In [68]:
from confluence_rag_integration import query_customer
# def query_customer(customer_id: str, question: str, top_k: int = 3):
#     """Query documents for a customer."""
#     manager = CustomerManager()
#     query_mgr = QueryManager(manager)
#     return query_mgr.query(customer_id, question, top_k)
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from typing import List

LLM_MODEL = "gemini-2.5-flash"
MODEL_PROVIDER = "google_genai"
customer_id = "acme_corp"
top_k = 5

# class RetrieveState(BaseModel):
#     query: str
#     preprocessed_query: List[str]
#     retrieved_documents: List[str]

from langchain_core.runnables import RunnableConfig

@tool
def retrieve_knowledge(query: str, config: RunnableConfig) -> List[str]:
    """
    Search and retrieve relevant documentation from Confluence knowledge base.
    Use this when you need to find information about products, procedures, 
    troubleshooting steps, or any documented knowledge to help resolve tickets.
    
    Args:
        query: The search query or question to find relevant documents
    Returns:
        A string of the retrieved knowledge.
    """
    result = query_customer(config.configurable["customer_id"], query, config.configurable["top_k"])
    formatted_results = []
    for doc in result.documents:
        source = doc.get("source", "Unknown")
        content = doc.get("content", "")
        formatted_results.append(f"Source: {source}:\n{content}")
    return formatted_results


In [69]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")



In [70]:
from typing import List, Annotated
import operator
from langchain_core.messages import BaseMessage
from pydantic import BaseModel
from langchain_core.messages import AIMessage

class QueryVariations(BaseModel):
    queries: List[str]

class RetrieveState(BaseModel):
    query: str
    preprocessed_query: List[str]
    retrieved_documents: Annotated[List[str], operator.add]
    messages: Annotated[List[BaseMessage], operator.add]

def preprocess_query(state: RetrieveState) -> List[str]:
    last_message = state.messages[-1].content
    prompt = f"""Given this support ticket query, generate 3 different search queries that would help find relevant documentations
    Original query: {last_message}
    
    Generate queries that:
    1. Extract key technical terms and product names
    2. Rephrase as a statement or different question form
    3. Include related terms or synonyms
    
    Return as JSON array of strings."""

    queries =llm.with_structured_output(QueryVariations).invoke(prompt).queries
    return {"preprocessed_query": queries}

def retrieve_documents(state: RetrieveState, config: RunnableConfig) -> str:
    seen_sources = set()
    formatted_results = []
    for query in state.preprocessed_query:
        # Call the query_customer function directly
        result = query_customer(
            customer_id=config["configurable"]["customer_id"], 
            question=query, 
            top_k=config["configurable"]["top_k"]
        )
        for doc in result.documents:
            source = doc.get("source", "Unknown")
            content = doc.get("content", "")
            doc_string = f"Source: {source}:\n{content}"
            if doc_string not in seen_sources:
                seen_sources.add(doc_string)
                formatted_results.append(doc_string)
    return {"retrieved_documents": formatted_results}

def synthesize_response(state: RetrieveState) -> str:
    original_query = state.messages[-1].content
    documents = state.retrieved_documents
    if not documents:
        return {"messages": [AIMessage(content="I couldn't find any relevant documentation for your query. Please provide more details or try rephrasing your question.")]}
    
    context = "\n\n".join(documents)
    prompt = f"""Based on the following documentation, provide a helpful response to resolve this support ticket.
        
        Original Query: {original_query}
        
        Relevant Documentation:
        {context}
        
        Provide a clear, step-by-step response that directly addresses the user's issue. If the documentation doesn't fully answer the question, acknowledge what information is missing."""
        
    response = llm.invoke(prompt)
    return {"messages": [AIMessage(content=response.content)]}

In [71]:
from langgraph.graph import StateGraph
from langgraph.graph import END
graph_builder = StateGraph(RetrieveState)

graph_builder.add_node("preprocess_query", preprocess_query)
graph_builder.add_node("retrieve_documents", retrieve_documents)  # Fixed: renamed from retrieve_knowledge
graph_builder.add_node("synthesize_response", synthesize_response)

graph_builder.set_entry_point("preprocess_query")
graph_builder.add_edge("preprocess_query", "retrieve_documents")
graph_builder.add_edge("retrieve_documents", "synthesize_response")
graph_builder.add_edge("synthesize_response", END)

graph = graph_builder.compile()

In [73]:
from langchain_core.messages import HumanMessage

customer_id = "acme_corp"
top_k = 5

input_message = "how to reset password?"
config = {"configurable": {"top_k": top_k, "customer_id": customer_id}}

initial_state = RetrieveState(
    messages=[HumanMessage(content=input_message)],
    query=input_message,
    preprocessed_query=[],
    retrieved_documents=[],
)

for step in graph.stream(
    initial_state,
    stream_mode="values",
    config=config
):
    # Print the message
    step["messages"][-1].pretty_print()
    
    # Print preprocessed queries if available
    if step.get("preprocessed_query"):
        print("\n📝 Preprocessed Queries:")
        for i, query in enumerate(step["preprocessed_query"], 1):
            print(f"  {i}. {query}")
    
    # Print retrieved documents if available (truncated)
    if step.get("retrieved_documents"):
        print(f"\n📚 Retrieved Documents: {len(step['retrieved_documents'])} found")
        for i, doc in enumerate(step["retrieved_documents"], 1):  # Show first 3
            # Truncate each document to first 200 characters
            truncated = doc[:200] + "..." if len(doc) > 200 else doc
            print(f"\n  Document {i}:")
            print(f"  {truncated}")
        if len(step["retrieved_documents"]) > 3:
            print(f"\n  ... and {len(step['retrieved_documents']) - 3} more documents")

================================ Human Message =================================

how to reset password?
================================ Human Message =================================

how to reset password?

📝 Preprocessed Queries:
  1. reset password
  2. password reset instructions
  3. forgot password change
================================ Human Message =================================

how to reset password?

📝 Preprocessed Queries:
  1. reset password
  2. password reset instructions
  3. forgot password change

📚 Retrieved Documents: 5 found

  Document 1:
  Source: data/customers/acme_corp/exports/Demo service project/Demo service project/Access & Security/Accounts & Passwords/Active Directory/Resetting your AD Password FAQs.md:
[Access & Security](../.....

  Document 2:
  Source: data/customers/acme_corp/exports/Demo service project/Demo service project/Access & Security/Accounts & Passwords/Business Systems (Single Sign-On))/Reset your Business Systems (SSO) Password....